# Multi-layers Neural Net on MNIST dataset

This is the third introduction on tensorflow for RVL224 students

In this section, you will build a 4 layers neural net, which can achive much better accuracy than a linear classifier does

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline


In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


## Define network properties

Define input node, output node and internal variables for weights and biases.

In [17]:
n_input    = 784 # MNIST data input (img shape: 28*28)
n_hidden_1 = 16 # 1st layer num neruons
n_hidden_2 = 16 # 2nd layer num neurons

n_classes  = 10 # MNIST total classes (0-9 digits)


# input and output nodes
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


dropout_keep_prob = tf.placeholder("float")


# define weights and biases and innitialize randomly using Gaussian normal function
stddev = 0.05
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Define computational graph for the neural net



In [18]:
layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
layer_1 = tf.nn.relu(layer_1)

layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
layer_2 = tf.nn.relu(layer_2)


layer_2 = tf.nn.dropout(layer_2, dropout_keep_prob)

layer_3 = tf.matmul(layer_2, weights['out']) + biases['out']

In [19]:
# PREDICTION
evidence = layer_3

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=evidence, labels=y)) 
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 
corr = tf.equal(tf.argmax(evidence, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.initialize_all_variables()


Instructions for updating:
Use `tf.global_variables_initializer` instead.


## training and testing

In [21]:
# PARAMETERS
training_epochs = 20
batch_size      = 32
display_step    = 2
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 0.8}
        sess.run(optm, feed_dict=feeds)
        feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 1.0}
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys, dropout_keep_prob: 1.0}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels, dropout_keep_prob: 1.0}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

Epoch: 001/020 cost: 0.417548855
TRAIN ACCURACY: 0.875
TEST ACCURACY: 0.897
Epoch: 003/020 cost: 0.302602040
TRAIN ACCURACY: 0.906
TEST ACCURACY: 0.915
Epoch: 005/020 cost: 0.250842789
TRAIN ACCURACY: 0.875
TEST ACCURACY: 0.927
Epoch: 007/020 cost: 0.226335125
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.931
Epoch: 009/020 cost: 0.212238196
TRAIN ACCURACY: 0.938
TEST ACCURACY: 0.934
Epoch: 011/020 cost: 0.200988480
TRAIN ACCURACY: 0.969
TEST ACCURACY: 0.935
Epoch: 013/020 cost: 0.193436090
TRAIN ACCURACY: 0.938
TEST ACCURACY: 0.938
Epoch: 015/020 cost: 0.187910496
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.939
Epoch: 017/020 cost: 0.182056193
TRAIN ACCURACY: 0.969
TEST ACCURACY: 0.940
Epoch: 019/020 cost: 0.177077322
TRAIN ACCURACY: 0.969
TEST ACCURACY: 0.937
OPTIMIZATION FINISHED


## exercise

Please add addtional layer of neural net, or increase the hidden layer size to see if you can achive up to 99% of accuracy.

Also feed some images in test dataset randomly and check if the neural net can predict correctly.